# Semantisk Kernel med OpenBnB MCP Server Integration

Den här anteckningsboken visar hur man använder Semantisk Kernel med den faktiska OpenBnB MCP-servern för att söka efter riktiga Airbnb-boenden med MCPStdioPlugin. För LLM-åtkomst används Azure AI Foundry. För att konfigurera dina miljövariabler kan du följa [Setup Lesson](/00-course-setup/README.md)


## Importera de nödvändiga paketen


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Skapa MCP Plugin-anslutningen

Vi kommer att ansluta till [OpenBnB MCP-servern](https://github.com/openbnb-org/mcp-server-airbnb) med MCPStdioPlugin. Den här servern erbjuder sökfunktionalitet för Airbnb via paketet @openbnb/mcp-server-airbnb.


## Skapa klienten

I det här exemplet kommer vi att använda Azure AI Foundry för vår LLM-åtkomst. Se till att dina miljövariabler är korrekt inställda.


## Miljökonfiguration

Konfigurera inställningar för Azure OpenAI. Se till att du har följande miljövariabler inställda:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Förstå OpenBnB MCP-integrationen

Den här notebooken ansluter till **den riktiga OpenBnB MCP-servern** som erbjuder faktisk sökfunktionalitet för Airbnb.

### Så här fungerar det:

1. **MCPStdioPlugin**: Använder standard in-/utgångskommunikation med MCP-servern
2. **Äkta NPM-paket**: Hämtar och kör `@openbnb/mcp-server-airbnb` via npx
3. **Live-data**: Returnerar verklig Airbnb-fastighetsdata från deras API:er
4. **Funktionsupptäckt**: Agenten upptäcker automatiskt tillgängliga funktioner från MCP-servern

### Tillgängliga funktioner:

OpenBnB MCP-servern erbjuder vanligtvis:
- **search_listings** - Sök efter Airbnb-fastigheter baserat på plats och kriterier
- **get_listing_details** - Hämta detaljerad information om specifika fastigheter
- **check_availability** - Kontrollera tillgänglighet för specifika datum
- **get_reviews** - Hämta recensioner för fastigheter
- **get_host_info** - Hämta information om fastighetsvärdar

### Förutsättningar:

- **Node.js** installerat på ditt system
- **Internetanslutning** för att ladda ner MCP-serverpaketet
- **NPX** tillgängligt (ingår med Node.js)

### Testa anslutningen:

Du kan testa MCP-servern manuellt genom att köra:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Detta kommer att ladda ner och starta OpenBnB MCP-servern, som Semantic Kernel sedan ansluter till för verklig Airbnb-data.


## Köra agenten med OpenBnB MCP-servern

Nu ska vi köra AI-agenten som ansluter till OpenBnB MCP-servern för att söka efter riktiga Airbnb-boenden i Stockholm för 2 vuxna och 1 barn. Du kan gärna ändra listan `user_inputs` för att modifiera sökkriterierna.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Sammanfattning
Grattis! Du har framgångsrikt byggt en AI-agent som integrerar med verkliga boendesökningar med hjälp av Model Context Protocol (MCP):

## Använda teknologier:
- Semantic Kernel - För att bygga intelligenta agenter med Azure OpenAI
- Azure AI Foundry - För LLM-funktioner och chattkomplettering
- MCP (Model Context Protocol) - För standardiserad verktygsintegration
- OpenBnB MCP Server - För verklig Airbnb-sökningsfunktionalitet
- Node.js/NPX - För att köra den externa MCP-servern

## Vad du har lärt dig:
- MCP-integration: Att koppla Semantic Kernel-agenter till externa MCP-servrar
- Realtidsdataåtkomst: Söka efter faktiska Airbnb-egendomar via live-API:er
- Protokollkommunikation: Använda stdio-kommunikation mellan agent och MCP-server
- Funktionsupptäckt: Automatiskt upptäcka tillgängliga funktioner från MCP-servrar
- Strömmande svar: Fånga och logga funktionsanrop i realtid
- HTML-rendering: Formatera agentens svar med stiliserade tabeller och interaktiva visningar

## Nästa steg:
- Integrera ytterligare MCP-servrar (väder, flyg, restauranger)
- Bygga ett multi-agent-system som kombinerar MCP- och A2A-protokoll
- Skapa anpassade MCP-servrar för dina egna datakällor
- Implementera ihågkommande konversationer över flera sessioner
- Distribuera agenten till Azure Functions med MCP-serverorkestrering
- Lägg till användarautentisering och bokningsfunktioner



---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, vänligen notera att automatiska översättningar kan innehålla fel eller felaktigheter. Det ursprungliga dokumentet på dess originalspråk bör betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för eventuella missförstånd eller feltolkningar som uppstår vid användning av denna översättning.
